# Library Import

In [2]:
# Ignore warnings

import pandas as pd
import numpy as np
from pathlib import Path

import analyzer
import neural_network as nn
import convolutional_neural_network as cnn
import visualizer

# Training Data Set Preparation

In [ ]:
resolution = [250,250]
# Resolution should be consistent throughout the file

In [ ]:

# Because the training dataset are individual, the list of each data just contain themselves.
data1 = analyzer.Data("MORE_DATA/db_Y_0027.okc", resolution) 
data2 = analyzer.Data("MORE_DATA/db_Y_0030.okc", resolution)


# Training The Model

In [ ]:
# Run this section if you want to use NN model

array1, headers1, non_nan_indices1, num_grids1 = nn.data_arranger(data1.df)
array2, headers2, non_nan_indices2, num_grids2 = nn.data_arranger(data2.df)

# The learning rate, batch size, and epochs are proven to be working.

nn_model = nn.model_create_compile(headers1, 0.05)

nn_model, loss_hist = nn.model_train(nn_model, array1, 1000, 10)
nn_model, loss_hist = nn.model_train(nn_model, array2, 1000, 10)

In [ ]:
# Run this section if you want to use CNN model

array1, headers1, indices1 = cnn.data_arranger(data1.df, resolution)
array2, headers2, indices2 = cnn.data_arranger(data2.df, resolution)

# The learning rate and epochs are proven to be working.

cnn_model = cnn.model_2D_create_compile(headers1, 0.05, resolution)

cnn_model, loss_hist = cnn.model_2D_train(cnn_model, array1, 3)
cnn_model, loss_hist = cnn.model_2D_train(cnn_model, array2, 3)

# Data Classification

Choose codes from these 3 codes below to run if you want to classify few individual data.

In [ ]:
data3 = analyzer.Data("MORE_DATA/db_Y_0049.okc", resolution)

In [ ]:
# Run this code if your model choice is NN
array3, headers3, non_nan_indices3, num_grids3 = nn.data_arranger(data3.df)
data3.df = nn.model_classification(nn_model, array3, non_nan_indices3, num_grids3, data3.df, False)

In [ ]:
# Run this code if your model choice is CNN
array3, headers3, indices3 = cnn.data_arranger(data3.df, resolution)
data3.df = cnn.model_2D_classification(cnn_model, array3, indices3, data3.df, False)

Choose codes from these 3 codes below to run if you want to classify a series of data.

In [ ]:
list_paths_classify = [f"MORE_DATA/db_Y_{i:04d}.okc" for i in range(99)]

Data_classify = [analyzer.Data(path, list_paths_classify, resolution) for path in list_paths_classify] 

In [ ]:
# Run this code if your model choice is NN
for data in Data_classify:
    array, headers, non_nan_indices, num_grids = nn.data_arranger(data.df)
    data.df = nn.model_classification(nn_model, array, non_nan_indices, num_grids, data.df, False)

In [ ]:
# Run this code if your model choice is CNN
for data in Data_classify:
    array, headers, indices = cnn.data_arranger(data.df, data.resolution)
    data.df = cnn.model_2D_classification(cnn_model, array, indices, data.df, False)

# Data Export

Choose codes from these 2 codes below to run if you want to classify few individual data.

In [ ]:
# Run this code to export the image
visualizer.plot_2D_df(data3.df, 'is_boundary', 'classification.png')

In [ ]:
# Run this code to export the csv file
data3.df.to_csv('classification.csv', index=False)

# Temporary Code Zone
---

### Merge 3D CSVs

In [ ]:
import os
import csv

path = '3D_DATA/small_db_result'
output_file = '3D_DATA/small_db.csv'

files = [file for file in os.listdir(path) if file.endswith('.csv')]

with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    writer = None
    
    for index, filename in enumerate(files):
        print(f"merging file: {index + 1}/{len(files)}", flush=True)
        
        with open(os.path.join(path, filename), 'r', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            
            header = next(reader)
            
            if writer is None:
                writer = csv.writer(outfile)
                writer.writerow(header)
            
            for row in reader:
                writer.writerow(row)

print("All files merged successfully!")


### Plot Streamline 2D

In [1]:
from visualizer import plot_3D_to_2D_slice_streamline

plot_3D_to_2D_slice_streamline(input_file="3D_DATA/ra10e7_result/ra10e7_100.csv", output_file="streamlines.html", direction='y', seed_points_resolution=[20,20], max_time=0.2, cmap = 'viridis', axis_limits=[-0.5,0.5,0,1])

### Plot Streamline 3D

✅ from a manually made velocity field, plot streamline (for test)

✅ generate a csv file from a set velocity field(for test)

✅ read from csv

In [1]:
import pandas as pd
import numpy as np
import pyvista as pv

# Read the CSV file into a DataFrame
df = pd.read_csv('test.csv')

points = df[['x', 'y', 'z']].values
velocities = df[['x_velocity', 'y_velocity', 'z_velocity']].values

# Extract unique coordinate values for each axis (ensure they are sorted)
x_vals = np.sort(df['x'].unique())
y_vals = np.sort(df['y'].unique())
z_vals = np.sort(df['z'].unique())

# Determine grid dimensions
nx, ny, nz = len(x_vals), len(y_vals), len(z_vals)

# Reshape coordinates
x = df['x'].values.reshape((nx, ny, nz))
y = df['y'].values.reshape((nx, ny, nz))
z = df['z'].values.reshape((nx, ny, nz))

# Create the StructuredGrid
grid = pv.StructuredGrid(x, y, z)

# Add the velocity vectors
grid.point_data['velocity'] = velocities

seed_x, seed_y, seed_z = np.meshgrid(
    np.linspace(-0.5, 0.5, 3), # min, max, num
    np.linspace(-0.5, 0.5, 3),
    np.linspace(-0.5, 0.5, 3)
    )
seed_x = seed_x.ravel()
seed_y = seed_y.ravel()
seed_z = seed_z.ravel()

seed_points = np.column_stack((seed_x, seed_y, seed_z))
seed = pv.PolyData(seed_points)

streamlines = grid.streamlines_from_source(
    source=seed,
    vectors='velocity',
    integration_direction='both',
    max_time=10,
    initial_step_length=0.01,
    terminal_speed=1e-3
)

velocity_vectors = streamlines['velocity']
velocity_magnitude = np.linalg.norm(velocity_vectors, axis=1)
streamlines['velocity_magnitude'] = velocity_magnitude

# Visualize and export streamlines as HTML
plotter = pv.Plotter(off_screen=True)
plotter.add_mesh(grid.outline(), color='k')


plotter.add_mesh(
    streamlines.tube(radius=0.01),
    scalars='velocity_magnitude',
    cmap='viridis',  # Use the colormap specified in the function argument
    scalar_bar_args={'title': 'Velocity Magnitude'}
)

plotter.view_isometric()
# Show grid with axis labels
plotter.show_grid(
    xtitle='X',
    ytitle='Y',
    ztitle='Z',
    grid='front'  # Display the grid in front of the scene
)

plotter.export_html('output_file.html')

read from multiple files

In [14]:
import pandas as pd
import numpy as np
import pyvista as pv
import glob
import os

# in such/a/folder/name_result, the CSVs will be like name_1.csv, name_2.csv, ...

# Read and concatenate all CSV files in the folder
folder_path = '3D_DATA/ra10e7_result'
# Extract the last part of the path and remove 'result'
base_name = os.path.basename(folder_path).replace('result', '')
# Create the search pattern
pattern = f"{base_name}*.csv"
# Get sorted list of matching CSV files
csv_files = sorted(glob.glob(os.path.join(folder_path, pattern)))
df_list = [pd.read_csv(f) for f in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Extract coordinates and velocity vectors
points = df[['x', 'y', 'z']].values
velocities = df[['x_velocity', 'y_velocity', 'z_velocity']].values


In [ ]:
# Get sorted unique coordinate values along each axis
x_vals = np.sort(df['x'].unique())
y_vals = np.sort(df['y'].unique())
z_vals = np.sort(df['z'].unique())

# Grid dimensions
nx, ny, nz = len(x_vals), len(y_vals), len(z_vals)

# Reshape coordinates to match structured grid shape
x = df['x'].values.reshape((nx, ny, nz))
y = df['y'].values.reshape((nx, ny, nz))
z = df['z'].values.reshape((nx, ny, nz))

# Create the structured grid
grid = pv.StructuredGrid(x, y, z)

# Attach velocity vectors as point data
grid.point_data['velocity'] = velocities

In [ ]:

seed_x, seed_y, seed_z = np.meshgrid(
    np.linspace(-0.5, 0.5, 3), # min, max, num
    np.linspace(-0.5, 0.5, 3),
    np.linspace(-0.5, 0.5, 3)
    )
seed_x = seed_x.ravel()
seed_y = seed_y.ravel()
seed_z = seed_z.ravel()

seed_points = np.column_stack((seed_x, seed_y, seed_z))
seed = pv.PolyData(seed_points)

streamlines = grid.streamlines_from_source(
    source=seed,
    vectors='velocity',
    integration_direction='both',
    max_time=10,
    initial_step_length=0.01,
    terminal_speed=1e-3
)

velocity_vectors = streamlines['velocity']
velocity_magnitude = np.linalg.norm(velocity_vectors, axis=1)
streamlines['velocity_magnitude'] = velocity_magnitude

# Visualize and export streamlines as HTML
plotter = pv.Plotter(off_screen=True)
plotter.add_mesh(grid.outline(), color='k')


plotter.add_mesh(
    streamlines.tube(radius=0.01),
    scalars='velocity_magnitude',
    cmap='viridis',  # Use the colormap specified in the function argument
    scalar_bar_args={'title': 'Velocity Magnitude'}
)

plotter.view_isometric()
# Show grid with axis labels
plotter.show_grid(
    xtitle='X',
    ytitle='Y',
    ztitle='Z',
    grid='front'  # Display the grid in front of the scene
)

plotter.export_html('output_file.html')